# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 6 2023 12:28PM,256725,12,HH-39815,Hush Hush,Released
1,Feb 6 2023 12:28PM,256725,12,HH-39816,Hush Hush,Released
2,Feb 6 2023 12:28PM,256725,12,HH-39817,Hush Hush,Released
3,Feb 6 2023 12:28PM,256725,12,HH-39818,Hush Hush,Released
4,Feb 6 2023 12:28PM,256725,12,HH-39819,Hush Hush,Released
5,Feb 6 2023 12:28PM,256725,12,HH-39820,Hush Hush,Released
6,Feb 6 2023 12:28PM,256725,12,HH-39821,Hush Hush,Released
7,Feb 6 2023 12:28PM,256725,12,HH-39822,Hush Hush,Released
8,Feb 6 2023 12:25PM,256724,21,640685,"NBTY Global, Inc.",Released
9,Feb 6 2023 12:17PM,256723,10,0086355390,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,256721,Released,36
29,256722,Released,45
30,256723,Released,33
31,256724,Released,1
32,256725,Released,8


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256721,NaN,NaN,36.0
256722,NaN,NaN,45.0
256723,NaN,NaN,33.0
256724,NaN,NaN,1.0
256725,NaN,NaN,8.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256656,1.0,1.0,1.0
256676,0.0,0.0,2.0
256683,0.0,0.0,1.0
256684,0.0,0.0,1.0
256685,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256656,1,1,1
256676,0,0,2
256683,0,0,1
256684,0,0,1
256685,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256656,1,1,1
1,256676,0,0,2
2,256683,0,0,1
3,256684,0,0,1
4,256685,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256656,1,1,1
1,256676,,,2
2,256683,,,1
3,256684,,,1
4,256685,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 6 2023 12:28PM,256725,12,Hush Hush
8,Feb 6 2023 12:25PM,256724,21,"NBTY Global, Inc."
9,Feb 6 2023 12:17PM,256723,10,ISDIN Corporation
42,Feb 6 2023 12:17PM,256722,10,ISDIN Corporation
87,Feb 6 2023 12:10PM,256721,10,ISDIN Corporation
123,Feb 6 2023 12:04PM,256719,19,"Innogenix, LLC"
145,Feb 6 2023 11:44AM,256718,10,Yusen – 3D Matrix
146,Feb 6 2023 11:43AM,256702,10,Bio-PRF
148,Feb 6 2023 11:42AM,256715,10,ISDIN Corporation
203,Feb 6 2023 11:42AM,256717,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 6 2023 12:28PM,256725,12,Hush Hush,,,8
1,Feb 6 2023 12:25PM,256724,21,"NBTY Global, Inc.",,,1
2,Feb 6 2023 12:17PM,256723,10,ISDIN Corporation,,,33
3,Feb 6 2023 12:17PM,256722,10,ISDIN Corporation,,,45
4,Feb 6 2023 12:10PM,256721,10,ISDIN Corporation,,,36
5,Feb 6 2023 12:04PM,256719,19,"Innogenix, LLC",,13,9
6,Feb 6 2023 11:44AM,256718,10,Yusen – 3D Matrix,,,1
7,Feb 6 2023 11:43AM,256702,10,Bio-PRF,,,2
8,Feb 6 2023 11:42AM,256715,10,ISDIN Corporation,,,55
9,Feb 6 2023 11:42AM,256717,10,ISDIN Corporation,,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 6 2023 12:28PM,256725,12,Hush Hush,8,,
1,Feb 6 2023 12:25PM,256724,21,"NBTY Global, Inc.",1,,
2,Feb 6 2023 12:17PM,256723,10,ISDIN Corporation,33,,
3,Feb 6 2023 12:17PM,256722,10,ISDIN Corporation,45,,
4,Feb 6 2023 12:10PM,256721,10,ISDIN Corporation,36,,
5,Feb 6 2023 12:04PM,256719,19,"Innogenix, LLC",9,13,
6,Feb 6 2023 11:44AM,256718,10,Yusen – 3D Matrix,1,,
7,Feb 6 2023 11:43AM,256702,10,Bio-PRF,2,,
8,Feb 6 2023 11:42AM,256715,10,ISDIN Corporation,55,,
9,Feb 6 2023 11:42AM,256717,10,ISDIN Corporation,2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 6 2023 12:28PM,256725,12,Hush Hush,8,,
1,Feb 6 2023 12:25PM,256724,21,"NBTY Global, Inc.",1,,
2,Feb 6 2023 12:17PM,256723,10,ISDIN Corporation,33,,
3,Feb 6 2023 12:17PM,256722,10,ISDIN Corporation,45,,
4,Feb 6 2023 12:10PM,256721,10,ISDIN Corporation,36,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 6 2023 12:28PM,256725,12,Hush Hush,8.0,NaN,NaN
1,Feb 6 2023 12:25PM,256724,21,"NBTY Global, Inc.",1.0,NaN,NaN
2,Feb 6 2023 12:17PM,256723,10,ISDIN Corporation,33.0,NaN,NaN
3,Feb 6 2023 12:17PM,256722,10,ISDIN Corporation,45.0,NaN,NaN
4,Feb 6 2023 12:10PM,256721,10,ISDIN Corporation,36.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 6 2023 12:28PM,256725,12,Hush Hush,8.0,0.0,0.0
1,Feb 6 2023 12:25PM,256724,21,"NBTY Global, Inc.",1.0,0.0,0.0
2,Feb 6 2023 12:17PM,256723,10,ISDIN Corporation,33.0,0.0,0.0
3,Feb 6 2023 12:17PM,256722,10,ISDIN Corporation,45.0,0.0,0.0
4,Feb 6 2023 12:10PM,256721,10,ISDIN Corporation,36.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3337249,204.0,5.0,0.0
12,1026831,9.0,2.0,0.0
15,256695,2.0,32.0,0.0
16,513391,1.0,1.0,0.0
19,1283478,17.0,14.0,1.0
21,1026776,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3337249,204.0,5.0,0.0
1,12,1026831,9.0,2.0,0.0
2,15,256695,2.0,32.0,0.0
3,16,513391,1.0,1.0,0.0
4,19,1283478,17.0,14.0,1.0
5,21,1026776,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,204.0,5.0,0.0
1,12,9.0,2.0,0.0
2,15,2.0,32.0,0.0
3,16,1.0,1.0,0.0
4,19,17.0,14.0,1.0
5,21,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,204.0
1,12,Released,9.0
2,15,Released,2.0
3,16,Released,1.0
4,19,Released,17.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21
Status,,,,,,
Completed,0.0,0.0,0.0,0.0,1.0,0.0
Executing,5.0,2.0,32.0,1.0,14.0,0.0
Released,204.0,9.0,2.0,1.0,17.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21
0,Completed,0.0,0.0,0.0,0.0,1.0,0.0
1,Executing,5.0,2.0,32.0,1.0,14.0,0.0
2,Released,204.0,9.0,2.0,1.0,17.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21
0,Completed,0.0,0.0,0.0,0.0,1.0,0.0
1,Executing,5.0,2.0,32.0,1.0,14.0,0.0
2,Released,204.0,9.0,2.0,1.0,17.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()